In [1]:
cd ..

c:\Users\Usuario\OneDrive\Documents\IE\3. Trimestre\Venture Lab & Capstone\Capstone\Tech side\vl_optimizer\equation_modeling


In [2]:
cd ..

c:\Users\Usuario\OneDrive\Documents\IE\3. Trimestre\Venture Lab & Capstone\Capstone\Tech side\vl_optimizer


Import required libraries

In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Data Center
## Code to generate the synthesized data required for the Data Center constraint formula:

Importing files

In [4]:
df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-09T19-25-47_industrial_site2_processed.csv')

Creating the target dataframe

In [5]:
data_center_target = df[['Time', '07 Data Center_Electric_Active Energy (kWh)']]
data_center_target.to_csv('data/target_variables/target_data_center.csv', index=False)

[Server Source](https://cc-techgroup.com/data-center-energy-consumption/#:~:text=this%20work%20efficiently.-,How%20Many%20kWh%20Does%20a%20Server%20Use%3F,to%201%2C900%20kWh%20every%20year.)

Average annual power may be around 1,800 to 1,900 kWh every year per server (divide by 365 then 24 the 4)   0.05279680365 kWh per 15 minutes

[Network Switch Source](https://www.genuinemodules.com/how-much-power-does-a-24-port-switch-use_a8539#:~:text=However%2C%20as%20a%20general%20estimate,requirements%20of%20the%20connected%20devices.)


However, as a general estimate, a non-PoE 24 port switch may consume around 20-50 watts of power, while a PoE-enabled switch may consume around 100-250 watts or more, depending on the power requirements of the connected devices. 
PoE: 25Wh - 62.5Wh per 15 minutes
Non-PoE: 5Wh - 12.5Wh per 15 minutes
    
 [HDD & SDD Source](https://dataspan.com/blog/how-much-energy-do-data-centers-use/#:~:text=Depending%20on%20disk%20size%2C%20an,around%206%20watts%20per%20disk.)   

Depending on disk size, an HDD can use anywhere from 6 to 9 watts at maximum capacity. While this is lower than in previous years, power requirements for SSDs have remained mostly constant at around 6 watts per disk.

HDD: 0.175Wh - 0.2625Wh per 15 minutes
SSD: 0.175Wh per 15 minutes


## Synthesized dataset containing the following columns:

total_data_center_consumption_kwh = β1*Number_of_active_servers + β2*Number_of_active_network_switches + β3*Number_of_active_hard_drives

	•	Number of active servers
	•	Number of active network switches (PoE)
	•	Number of active network switches (Non-PoE)
	•	Number of active hard drives (HDD)
	•	Number of active solid-state drives (SSD)
	•	Total energy consumption in kWh

In [6]:
# Load the CSV file
df = pd.read_csv('data/target_variables/target_data_center.csv')

# Ensure '07 Data Center_Electric_Active Energy (kWh)' is numeric
df['07 Data Center_Electric_Active Energy (kWh)'] = pd.to_numeric(df['07 Data Center_Electric_Active Energy (kWh)'], errors='coerce')

# Drop rows with NaN values in the target consumption column
df.dropna(subset=['07 Data Center_Electric_Active Energy (kWh)'], inplace=True)

# Define the power consumption per hour for each component
power_consumption_per_hour = {
    'server': 0.2111872146,  # kWh per hour
    # 'network_switch': (0.1748 + 0.035) / 2,  # kWh per hour
    'network_switch_poe': (((100 / 1000 ) + (249.6 / 1000))/2),  # Convert Wh to kWh per hour
    'network_switch_non_poe': (((20 / 1000 ) + (50 / 1000))/2),  # Convert Wh to kWh per hour
    'hdd': (((0.7 / 1000 ) + (1.05 / 1000))/2),  # Convert Wh to kWh per hour
    'ssd': 0.7 / 1000  # Convert Wh to kWh per hour
}

# List of component names
components = list(power_consumption_per_hour.keys())

# Function to calculate total consumption based on the number of active units
def calculate_consumption(x):
    total_consumption = 0
    for i, component in enumerate(components):
        total_consumption += x[i] * power_consumption_per_hour[component]
    return total_consumption

# Function to minimize: difference between calculated and target consumption
def objective(x, target_consumption):
    calculated = calculate_consumption(x)
    return np.abs(calculated - target_consumption)

# List to store synthesized data
synthesized_data = []

# Optimize for each hour in the dataset
for index, row in df.iterrows():
    target_consumption = row['07 Data Center_Electric_Active Energy (kWh)']
    
    # Initial guess: random integers within a reasonable range
    initial_guess = np.random.randint(1, 10, size=len(components))
    
    # Bounds: each component should be at least 1
    bounds = [(1, None)] * len(components)
    
    # Optimize
    result = minimize(objective, initial_guess, args=(target_consumption,), bounds=bounds, method='L-BFGS-B')
    
    # Round the result to the nearest whole number
    rounded_result = np.round(result.x).astype(int)
    
    # Append to the synthesized data
    synthesized_data.append(rounded_result)

# Create a DataFrame for the synthesized data
synthesized_df = pd.DataFrame(synthesized_data, columns=components)
synthesized_df['total_consumption'] = synthesized_df.apply(lambda row: calculate_consumption(row), axis=1)
synthesized_df['target_consumption'] = df['07 Data Center_Electric_Active Energy (kWh)'].values
synthesized_df['Time'] = df['Time'].values

# Display the first few rows of the synthesized data
synthesized_df.head()

,server,network_switch_poe,network_switch_non_poe,hdd,ssd,total_consumption,target_consumption,Time
0,6,6,5,3,7,2.498448,2.575,2022-07-01 00:00:00
1,8,4,1,7,1,2.430523,2.500,2022-07-01 01:00:00
2,8,4,1,9,4,2.434373,2.550,2022-07-01 02:00:00
3,6,6,5,2,4,2.495473,2.500,2022-07-01 03:00:00
4,5,6,10,1,2,2.457011,2.550,2022-07-01 04:00:00


Save synthesized data to CSV

In [7]:
output_path = 'data/synthesized_data/data_center_synthesized.csv'
synthesized_df.to_csv(output_path, index=False)